## Pseudonym error performance under multiple interference

In [1227]:
import json
import pandas as pd
import math
import numpy as np
from scipy import signal
import h5py
import matplotlib.pyplot as plt
import scipy.signal as signal
import statistics as stats
import scipy.io
from scipy.spatial.distance import hamming
import os


In [1228]:
def get_time_string(timestamp):
    '''
    Helper function to get data and time from timestamp
    INPUT: timestamp
    OUTPUT: data and time. Example: 01-04-2023, 19:50:27
    '''
    date_time = datetime.datetime.fromtimestamp(int(timestamp))
    return date_time.strftime("%m-%d-%Y, %H:%M:%S")

In [1229]:
def JsonLoad(folder, json_file):
    '''
    Load parameters from the saved json file
    INPUT
    ----
        folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
        json_file: the json file with all the specifications. Example: '/save_iq_w_tx_gold.json'
    OUTPUT
    ----
        samps_per_chip: samples per chip
        wotxrepeat: number of repeating IQ sample collection w/o transmission. Used as an input to 
        traverse_dataset() func
        rxrate: sampling rate at the receiver side
    '''
    #config_file = folder+'/'+json_file
    #config_file = ""+"/"+str(folder)+"/save_iq_w_tx_file.json"
    config_dict = json.load(open(json_file))[0]
    nsamps = config_dict['nsamps']
    rxrate = config_dict['rxrate']
    rxfreq = config_dict['rxfreq']
    wotxrepeat = config_dict['wotxrepeat']
    rxrepeat = config_dict['rxrepeat']
    txnodes = config_dict['txclients']
    rxnodes = config_dict['rxclients']

    return rxrepeat, rxrate, txnodes, rxnodes

In [1230]:
def traverse_dataset(meas_folder):
    '''
    Load data from hdf5 format measurement file
    INPUT
    ----
        meas_folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
    OUTPUT
    ----
        data: Collected IQ samples w/ transmission. It is indexed by the transmitter name
        noise: Collected IQ samples w/o transmission. It is indexed by the transmitter name
        txrxloc: transmitter and receiver names
    '''
    data = {}
    noise = {}
    txrxloc = {}

    dataset = h5py.File(meas_folder + '/measurements.hdf5', "r") #meas_folder
    #print("Dataset meta data:", list(dataset.attrs.items()))
    for cmd in dataset.keys():
        #print("Command:", cmd)
        if cmd == 'saveiq':
            cmd_time = list(dataset[cmd].keys())[0]
           # print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for rx_gain in dataset[cmd][cmd_time].keys():
               # print("   RX gain:", rx_gain)
                for rx in dataset[cmd][cmd_time][rx_gain].keys():
                    print("")
                    #print("     RX:", rx)
                    #print("       Measurement items:", list(dataset[cmd][cmd_time][rx_gain][rx].keys()))
        elif cmd == 'saveiq_w_tx':
            cmd_time = list(dataset[cmd].keys())[0]
            #print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for tx in dataset[cmd][cmd_time].keys():
                #print("   TX:", tx)
                
                if tx == 'wo_tx':
                    for rx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("       RX gain:", rx_gain)
                       # print(dataset[cmd][cmd_time][tx][rx_gain].keys())
                        for rx in dataset[cmd][cmd_time][tx][rx_gain].keys():
                            #print("         RX:", rx)
                            #print("           Measurement items:", list(dataset[cmd][cmd_time][tx][rx_gain][rx].keys()))
                            repeat = np.shape(dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'])[0]
                            #print("         repeat", repeat)

                            samplesNotx =  dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'][:repeat, :]
                            namelist = rx.split('-')
                            noise[namelist[1]] = samplesNotx
                else:
                    for tx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("     TX gain:", tx_gain)
                        for rx_gain in dataset[cmd][cmd_time][tx][tx_gain].keys():
                            #print("       RX gain:", rx_gain)
                            #print(dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys())
                            for rx in dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys():
                                repeat = np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'])[0]
                                #print("         RX:", rx, "; samples shape", np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples']))
                                #print("         Measurement items:", list(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx].keys()))
                                # print("         rxloc", (dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxloc'][0]))
                                # peak avg check
                                
                                txrxloc.setdefault(tx, []).extend([rx]*repeat)
                                rxsamples = dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'][:repeat, :]
                                data.setdefault(tx, []).append(np.array(rxsamples))

        else:                       
            print('Unsupported command: ', cmd)

    return data, noise, txrxloc

In [1231]:
def plotOnePSDForEachLink(rx_data, txrxloc, samp_rate=2e6, repeats=10):
    for txname in rx_data:
        print(txname)
        for i in range(0, len(rx_data[txname]), repeats):
            plt.figure()
            plt.psd(rx_data[txname][i][1], Fs = samp_rate/1000)
            #plt.ylim(-110, -60)
            #plt.yticks(ticks=[-110, -100, -90, -80, -70, -60])
            plt.grid('on')
            plt.title('TX: {} RX: {}'.format(txname, txrxloc[txname][i]))
            plt.xlabel('Frequency (kHz)')
            plt.tight_layout()
            plt.show()

In [1232]:
# PURPOSE: perform preamble synchronization
#          Uses the (complex-valued) preamble signal. The cross-correlation 
#          of the preamble signal and the received signal (at the time
#          when the preamble is received) should have highest magnitude
#          at the index delay where the preamble approximately starts.  
# INPUT:   rx0: received signal (with a frequency offset)
#          preambleSignal: complex, known, transmitted preamble signal 
# OUTPUT:  lagIndex: the index of rx0 where the preamble signal has highest 
#              cross-correlation
#
def crossCorrelationMax(rx0, preambleSignal):

    # Cross correlate with the preamble to find it in the noisy signal
    lags      = signal.correlation_lags(len(rx0), len(preambleSignal), mode='valid')
    xcorr_out = signal.correlate(rx0, preambleSignal, mode='valid')
    xcorr_mag = np.abs(xcorr_out)
    # Don't let it sync to the end of the packet.
    packetLenSamples = 13440800
    maxIndex = np.argmax(xcorr_mag[:len(xcorr_mag)-packetLenSamples])
    lagIndex = lags[maxIndex]
    
#     print('The old lag ' + str(lagIndex))
    # Try to use the second lagIndex if the first one is not strong enough
    packetLenSamples = 0
    maxIndex2 = np.argmax(xcorr_mag[:len(xcorr_mag)-packetLenSamples])
    lagIndex2 = lags[maxIndex2]
    
# #     print("xcorr_mag[maxIndex2]", xcorr_mag[maxIndex2])
    
    if abs(xcorr_mag[maxIndex2]) > abs(xcorr_mag[maxIndex]):
        lagIndex = lags[maxIndex2] - Frame_length
        print('The new lag ' + str(lagIndex))
    else:
        print('The old lag ' + str(lagIndex))
    #Plot the selected signal.
    plt.figure(1)
    fig, subfigs = plt.subplots(2,1)
    subfigs[0].plot(np.real(rx0), label='Real RX Signal')
    subfigs[0].plot(np.imag(rx0), label='Imag RX Signal')
    scale_factor = np.mean(np.abs(rx0))/np.mean(np.abs(preambleSignal))
    subfigs[0].plot(range(lagIndex, lagIndex + len(preambleSignal)), scale_factor*np.real(preambleSignal), label='Preamble')
    #subfigs[0].legend()
    subfigs[1].plot(lags, xcorr_mag, label='|X-Correlation|')
    plt.xlabel('Sample Index', fontsize=14)
    plt.show()
    #plt.tight_layout()

    return lagIndex

In [1233]:
def text2bits(message):
    # Convert to characters of '1' and '0' in a vector.
    temp_message = []
    final_message = []
    for each in message:
        temp_message.append(format(ord(each), '07b'))
    for every in temp_message:
        for digit in every:
            final_message.append(int(digit))
    return final_message

In [1234]:
# def Calculate_mean(x):
#     swap = np.zeros(49,)
#     for i in range(49):
#         swap[i] = sum(abs(x[i*N:(i+1)*N])**2)
#         print('Received power:',swap[i])
#     return np.median(swap)

In [1235]:
import numpy as np
import math
def binvector2str(binvector):
    #binvector = binvector[0]
    length = len(binvector)
    eps = np.finfo('float').eps
    if abs(length/7 - round(length/7)) > eps:
        print('Length of bit stream must be a multiple of 7 to convert to a string.')
    # Each character requires 7 bits in standard ASCII
    num_characters = round(length/7)
    # Maximum value is first in the vector. Otherwise would use 0:1:length-1
    start = 6
    bin_values = []
    while start >= 0:
        bin_values.append(int(math.pow(2,start)))
        start = start - 1
    bin_values = np.array(bin_values)
    bin_values = np.transpose(bin_values)
    str_out = '' # Initialize character vector
    for i in range(num_characters):
        single_char = binvector[i*7:i*7+7]
        value = 0
        for counter in range(len(single_char)):
            value = value + (int(single_char[counter]) * int(bin_values[counter]))
        str_out += chr(int(value))
    return str_out

In [1236]:
# PURPOSE: Convert binary data to M-ary by making groups of log2(M)
#          bits and converting each bit to one M-ary digit.
# INPUT: Binary digit vector, with length as a multiple of log2(M)
# OUTPUT: M-ary digit vector
def binary2mary(data, M):

    log2M   = round(np.log2(M))
    # integer number of bits per group
    if (len(data) % log2M) != 0:
        print('Input to binary2mary must be divisible by log2(m).')
    data.shape = (len(data)//log2M, log2M)
    binaryValuesArray = 2**np.arange(log2M)
    marydata = data.dot(binaryValuesArray)
    return marydata

In [1237]:
# PURPOSE: convert input data stream to signal space values for
#          a particular modulation type (as specified by the inputVec
#          and outputVec).
# INPUT: data (groups of bits)
# OUTPUT: signal space values
def lut(data, inputVec, outputVec):
    if len(inputVec) != len(outputVec):
        print('Input and Output vectors must have identical length')
    # Initialize output
    output = np.zeros(data.shape)
    # For each possible data value
    eps = np.finfo('float').eps
    for i in range(len(inputVec)):
        # Find the indices where data is equal to that input value
        for k in range(len(data)):
            if abs(data[k]-inputVec[i]) < eps:
                # Set those indices in the output to be the appropriate output value.
                output[k] = outputVec[i]
    return output

# Pseudonym Decoding Below

In [1239]:
pseudonym_mesage = 'STOP'

pseudonym_packet = text2bits(pseudonym_mesage)
pseudonym_length = len(pseudonym_packet)

In [1240]:
'''
Protocol parameters!!!
'''
FFT = 64 # FFT size for extracting IQ sample in each subcarrier
OFDM_size = 80 # OFDM symbol with cyclic prefix
data_size = 48 # data_subcarriers
# mess_length = 560
Frame_length = 13440800
packet = 6000 # number of bits per pseudonym bit
samples = packet//10 # number of samples per chip
CP = 16
repeat =10
actual_message = 'I have done my PhD in electrical engineering at WashU. What a journey it has been!'
len(actual_message)

82

In [1241]:
CP = FFT//4  # 25% cyclic prefix
pilotValue = 2.83+2.83j # known values for pilots
pseudonymValue = 1.4142+1.4142j

In [1242]:
dataCarriers = np.array([-26,-25,-24,-23,-22,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-6,
                    -5,-4,-3,-2,-1,0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25])
pseudonymCarrier = np.array([26])
pilotCarriers = np.array([-21,-7,7,21])
guardCarriers = np.array([-32,-31,-30,-29,-28,-27,27,28,29,30,31])

In [1243]:
# Generate HTSTF signals for time synchronization
def Generate_HTSTF():
    data = scipy.io.loadmat('HTSTF.mat')
    new_data = data['stf'].reshape((1,len(data['stf'])))
    preamble = np.tile(new_data,10)[0]
    return preamble

In [1244]:
## FFT for data of window size=64
def DFT_data(x, n =64):
    return np.fft.fft(x,n)

In [1245]:
''' Calculates FFT to recover the frequency domain signal in subcarriers and stores them in a matrix '''

def subchannel_data(x):
    sort_data = np.zeros((FFT, len(x)//OFDM_size), dtype = np.complex64)
    for i in range(len(x)//OFDM_size):
        y = x[i*OFDM_size:(i+1)*OFDM_size]
        sort_data[:,i] = np.fft.fftshift(DFT_data(y[CP:],n=64))  
    return sort_data

In [1246]:
## calculate the power in each pseudonym bit and store it in matrix
def Cal_subch_power(x):
    sub_power = np.zeros((FFT,1))
    for i in range(FFT):
        sub_power[i] = sum(abs(x[i,:])**2)
    return sub_power

In [1247]:
'''
calculates the distance between two arrays.
'''
def Distance(X,Y):
    count = 0
    for i in range(len(X)):
        if X[i]!= Y[i]:
            count +=1
    return count

In [1248]:
## Make p-bit decisions by comparing patterns on bit-0 and bit-1
## Trace changes in power with the p-bit and compare it with the known chip pattern.
## This algorithm improves pseudonym detection in the presence of non-Gaussion type interferences

def Matched_Filter_Pseudonym_Detection_Algorithm(x):
    matching_filter0 =np.array([-1,1,-1,1,-1,1,-1,1,-1,1])
    matching_filter1 =np.array([1,-1,1,-1,1,-1,1,-1,1,-1])
    p_bit = []
    for i in range(28):
        pbit_data = x[i*OFDM_size*packet:(i+1)*packet*OFDM_size] # slices samples into one p-bit data = 6000 samples
        
        power = []
        Cr = []       
        threshold = 0.0
        quantization_level = np.array([1,-1])
        for j in range(10):
            chip_data = pbit_data[j*OFDM_size*samples:(j+1)*OFDM_size*samples]
            FFT_matrix = subchannel_data(chip_data)
            subchannel_power = Cal_subch_power(FFT_matrix)
            power.append(subchannel_power[48,0])
       
        # plt.plot(power)
        # plt.show()

        for k in range(1,10):
            if power[k] > power[k-1]:
                Cr.append(quantization_level[0])
            else:
                Cr.append(quantization_level[1])
            
        if np.dot(Cr,matching_filter1[1:]) >= np.dot(Cr,matching_filter0[1:]): # p-bit decision done by comparing p-bit powers with the threshold.
            p_bit.append(1)
        else:
            p_bit.append(0)   
    return np.array(p_bit)

In [1249]:
def Extract_Folders(x):
    r = []
    for root, dirs, files in os.walk(x):
        r.append(dirs)
    return r

In [1250]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''
def Average_Pseudonym_Error(x):
    # Load parameters from the JSON file which describe what was measured
 
    #folder = x
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
    samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    rxloc = 'cbrssdr1-browning-comp'
    
    #Calculate SNR
    Noise = rx_noise['browning'][0]
    noise_power = sum(abs(Noise)**2)/len(Noise)
#     print("Noise Power:",noise_power)
    rx_data[txloc] = np.vstack(rx_data[txloc])
    rxloc_arr = np.array(txrxloc[txloc])       

    # initialize error
    pseudonym_BER = 0
    P_s = 0
    for i in range(repeat):
        repNum = i
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][repNum]
    

        # synchronize pseudonym using preamble signal
        preambleSignal = Generate_HTSTF()
        lagIndex = crossCorrelationMax(rx0,preambleSignal)
        
        start = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start:]
               
        pseudonym_estimate = Matched_Filter_Pseudonym_Detection_Algorithm(Rx_signal)
        print('Detected Pseudonym:',binvector2str(pseudonym_estimate), '\n')
        repeat_BER = Distance(pseudonym_estimate,pseudonym_packet)
        if  repeat_BER !=0:
            print("Errors Found:",repeat_BER)
        print("Repeat Number:", i, "Number of Errors:",repeat_BER)
        pseudonym_BER += repeat_BER
 
        P_s += sum(abs(Rx_signal)**2)/len(Rx_signal)
    
    signal_power = P_s/repeat
    
    return pseudonym_BER, signal_power, noise_power

In [1251]:
'''
Computes the average pseudonym error in the experiment.
Experiment data is stored in folders that contain 10 repeats each.
'''
def Prob_Pseudonym_Error(x):
    folders = Extract_Folders(x)[0]
    num_folders = len(folders)
    print('Number of folders:',num_folders)
    pseudonym_BER = 0
    signal,noise = 0,0
    noise_power = []
    Exp_power = []
    for i in range(num_folders):
        print(folders[i])
        error,s,n= Average_Pseudonym_Error(x + '/'+folders[i])
        
        pseudonym_BER += error
        signal += s
        noise += n
        noise_power.append(n)

    Eb_No = 10*(np.log10(0.5*(signal-noise)/noise))
    
    print("total number of pseudonym bit errors:", pseudonym_BER)
    return pseudonym_BER/(repeat*num_folders*pseudonym_length), Eb_No, noise_power

In [1252]:
#x,y,AWGN = Prob_Pseudonym_Error("10dB_gain")
x,y,AWGN_RFI = Prob_Pseudonym_Error("data")

Number of folders: 2
Shout_meas_08-11-2024_14-33-42
The old lag 1843180
Detected Pseudonym: STOP 

Repeat Number: 0 Number of Errors: 0
The old lag 13014300
Detected Pseudonym: STOP 

Repeat Number: 1 Number of Errors: 0
The old lag 7011323
Detected Pseudonym: h)J' 

Errors Found: 19
Repeat Number: 2 Number of Errors: 19
The old lag 10475180
Detected Pseudonym: STOP 

Repeat Number: 3 Number of Errors: 0
The old lag 10205596
Detected Pseudonym: STOP 

Repeat Number: 4 Number of Errors: 0
The new lag 8935948
Detected Pseudonym: STOP 

Repeat Number: 5 Number of Errors: 0
The new lag 9666396
Detected Pseudonym: STOP 

Repeat Number: 6 Number of Errors: 0
The old lag 8396700
Detected Pseudonym: STOP 

Repeat Number: 7 Number of Errors: 0
The new lag 9127212
Detected Pseudonym: STOP 

Repeat Number: 8 Number of Errors: 0
The old lag 8857596
Detected Pseudonym: STOP 

Repeat Number: 9 Number of Errors: 0
Shout_meas_08-11-2024_13-59-45
The new lag 5484929
Detected Pseudonym: R>B 

Errors Fo

In [1253]:
print('SNR:',y,'\n Pseudonym_BER:',x)

SNR: -17.650730068980454 
 Pseudonym_BER: 0.10535714285714286
